In [76]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder,MinMaxScaler,StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier ,RandomForestClassifier ,GradientBoostingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier 
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import Ridge,Lasso
from sklearn.metrics import roc_auc_score ,mean_squared_error,accuracy_score,classification_report,roc_curve,confusion_matrix,f1_score
import warnings
warnings.filterwarnings('ignore')
from scipy.stats.mstats import winsorize
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split

In [77]:
pd.set_option('display.max_columns',None)

In [78]:
import os
for dirname, _, filenames in os.walk('../123'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

../123\LoanInterest.ipynb
../123\sample_submission.csv
../123\submission.csv
../123\test.csv
../123\train.csv
../123\Untitled.ipynb
../123\.ipynb_checkpoints\LoanInterest-checkpoint.ipynb
../123\.ipynb_checkpoints\Untitled-checkpoint.ipynb


In [79]:
train_data=pd.read_csv('../123/train.csv')

In [80]:
test_data=pd.read_csv('../123/test.csv')

In [81]:
train_data.head()

,Loan_ID,Loan_Amount_Requested,Length_Employed,Home_Owner,Annual_Income,Income_Verified,Purpose_Of_Loan,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Gender,Interest_Rate
0,10000001,"7,000",< 1 year,Rent,68000.0,not verified,car,18.37,0,NaN,9,14,Female,1
1,10000002,"30,000",4 years,Mortgage,NaN,VERIFIED - income,debt_consolidation,14.93,0,17.0,12,24,Female,3
2,10000003,"24,725",7 years,Mortgage,75566.4,VERIFIED - income source,debt_consolidation,15.88,0,NaN,12,16,Male,3
3,10000004,"16,000",< 1 year,NaN,56160.0,VERIFIED - income source,debt_consolidation,14.34,3,NaN,16,22,Male,3
4,10000005,"17,000",8 years,Own,96000.0,VERIFIED - income source,debt_consolidation,22.17,1,NaN,19,30,Female,1


In [82]:
train_data.shape

(164309, 14)

In [83]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 164309 entries, 0 to 164308
Data columns (total 14 columns):
Loan_ID                    164309 non-null int64
Loan_Amount_Requested      164309 non-null object
Length_Employed            156938 non-null object
Home_Owner                 138960 non-null object
Annual_Income              139207 non-null float64
Income_Verified            164309 non-null object
Purpose_Of_Loan            164309 non-null object
Debt_To_Income             164309 non-null float64
Inquiries_Last_6Mo         164309 non-null int64
Months_Since_Deliquency    75930 non-null float64
Number_Open_Accounts       164309 non-null int64
Total_Accounts             164309 non-null int64
Gender                     164309 non-null object
Interest_Rate              164309 non-null int64
dtypes: float64(3), int64(5), object(6)
memory usage: 17.6+ MB


In [84]:
train_data.isnull().sum()

Loan_ID                        0
Loan_Amount_Requested          0
Length_Employed             7371
Home_Owner                 25349
Annual_Income              25102
Income_Verified                0
Purpose_Of_Loan                0
Debt_To_Income                 0
Inquiries_Last_6Mo             0
Months_Since_Deliquency    88379
Number_Open_Accounts           0
Total_Accounts                 0
Gender                         0
Interest_Rate                  0
dtype: int64

In [85]:
# Function to identify the number of missing values in every feature
def missing_data(dataset):
    total = dataset.isnull().sum().sort_values(ascending=False)
    percent = (dataset.isnull().sum()/dataset.isnull().count()*100).sort_values(ascending=False)
    missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    return missing_data
    
    
missing_data = missing_data(train_data)
print(missing_data)

                         Total    Percent
Months_Since_Deliquency  88379  53.788289
Home_Owner               25349  15.427639
Annual_Income            25102  15.277313
Length_Employed           7371   4.486060
Interest_Rate                0   0.000000
Gender                       0   0.000000
Total_Accounts               0   0.000000
Number_Open_Accounts         0   0.000000
Inquiries_Last_6Mo           0   0.000000
Debt_To_Income               0   0.000000
Purpose_Of_Loan              0   0.000000
Income_Verified              0   0.000000
Loan_Amount_Requested        0   0.000000
Loan_ID                      0   0.000000


In [86]:
def check_datatypes(dataset):
    return dataset.dtypes

In [87]:
check_datatypes(train_data)

Loan_ID                      int64
Loan_Amount_Requested       object
Length_Employed             object
Home_Owner                  object
Annual_Income              float64
Income_Verified             object
Purpose_Of_Loan             object
Debt_To_Income             float64
Inquiries_Last_6Mo           int64
Months_Since_Deliquency    float64
Number_Open_Accounts         int64
Total_Accounts               int64
Gender                      object
Interest_Rate                int64
dtype: object

In [88]:
train_data.describe()

,Loan_ID,Annual_Income,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Interest_Rate
count,1.643090e+05,1.392070e+05,164309.000000,164309.000000,75930.000000,164309.000000,164309.000000,164309.000000
mean,1.008216e+07,7.333116e+04,17.207189,0.781698,34.229356,11.193818,25.067665,2.158951
std,4.743207e+04,6.037750e+04,7.845083,1.034747,21.761180,4.991813,11.583067,0.738364
min,1.000000e+07,4.000000e+03,0.000000,0.000000,0.000000,0.000000,2.000000,1.000000
25%,1.004108e+07,4.500000e+04,11.370000,0.000000,16.000000,8.000000,17.000000,2.000000
50%,1.008216e+07,6.300000e+04,16.840000,0.000000,31.000000,10.000000,23.000000,2.000000
75%,1.012323e+07,8.869750e+04,22.780000,1.000000,50.000000,14.000000,32.000000,3.000000
max,1.016431e+07,7.500000e+06,39.990000,8.000000,180.000000,76.000000,156.000000,3.000000


In [89]:
train_data.drop(columns={'Loan_ID'},inplace=True)

In [90]:
test_data.drop(columns={'Loan_ID'},inplace=True)

In [91]:
gender_df=pd.get_dummies(train_data['Gender'])

In [92]:
gender_test=pd.get_dummies(test_data['Gender'])

In [93]:
train_data['Female'] = gender_df['Female']
train_data['Male']=gender_df['Male']


In [94]:
test_data['Female'] = gender_test['Female']
test_data['Male']=gender_test['Male']


In [95]:
train_data.head()

,Loan_Amount_Requested,Length_Employed,Home_Owner,Annual_Income,Income_Verified,Purpose_Of_Loan,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Gender,Interest_Rate,Female,Male
0,"7,000",< 1 year,Rent,68000.0,not verified,car,18.37,0,NaN,9,14,Female,1,1,0
1,"30,000",4 years,Mortgage,NaN,VERIFIED - income,debt_consolidation,14.93,0,17.0,12,24,Female,3,1,0
2,"24,725",7 years,Mortgage,75566.4,VERIFIED - income source,debt_consolidation,15.88,0,NaN,12,16,Male,3,0,1
3,"16,000",< 1 year,NaN,56160.0,VERIFIED - income source,debt_consolidation,14.34,3,NaN,16,22,Male,3,0,1
4,"17,000",8 years,Own,96000.0,VERIFIED - income source,debt_consolidation,22.17,1,NaN,19,30,Female,1,1,0


In [96]:
# Function to detect outliers in every feature
def detect_outliers(dataframe):
    cols = list(dataframe)
    outliers = pd.DataFrame(columns=['Feature','Number of Outliers'])
    
    for column in cols:
        if column in dataframe.select_dtypes(include=np.number).columns:
            # first quartile (Q1)
            q1 = dataframe[column].quantile(0.25) 
            
            # third quartile (Q3)
            q3 = dataframe[column].quantile(0.75)
            
            # IQR
            iqr = q3 - q1
            
            fence_low = q1 - (1.5*iqr)
            fence_high = q3 + (1.5*iqr)
            outliers = outliers.append({'Feature':column,'Number of Outliers':dataframe.loc[(dataframe[column] < fence_low) | (dataframe[column] > fence_high)].shape[0]},ignore_index=True)
    return outliers

detect_outliers(train_data)

,Feature,Number of Outliers
0,Annual_Income,5881
1,Debt_To_Income,12
2,Inquiries_Last_6Mo,12276
3,Months_Since_Deliquency,30
4,Number_Open_Accounts,3754
5,Total_Accounts,3099
6,Interest_Rate,0
7,Female,0
8,Male,0


In [97]:
train_data['Interest_Rate'].skew()

-0.260932482982295

In [98]:
owner_df=pd.get_dummies(train_data['Home_Owner'])

In [99]:
owner_test=pd.get_dummies(test_data['Home_Owner'])

In [100]:
train_data['Mortgage'] = owner_df['Mortgage']
train_data['Own']=owner_df['Own']
train_data['Rent']=owner_df['Rent']
train_data['Other']=owner_df['Other']
train_data['None']=owner_df['None']

In [101]:
test_data['Mortgage'] = owner_test['Mortgage']
test_data['Own']=owner_test['Own']
test_data['Rent']=owner_test['Rent']
test_data['Other']=owner_test['Other']
test_data['None']=owner_test['None']

In [102]:
verfified_income_df=pd.get_dummies(train_data['Income_Verified'])

In [103]:
verfified_income_test=pd.get_dummies(test_data['Income_Verified'])

In [104]:
train_data['Verified_Income'] = verfified_income_df['VERIFIED - income']
train_data['Verified_Income_Source']=verfified_income_df['VERIFIED - income source']
train_data['Not_Verified']=verfified_income_df['not verified']

In [105]:
test_data['Verified_Income'] = verfified_income_test['VERIFIED - income']
test_data['Verified_Income_Source']=verfified_income_test['VERIFIED - income source']
test_data['Not_Verified']=verfified_income_test['not verified']

In [106]:
train_data.drop(columns={'Income_Verified','Gender','Home_Owner'},inplace=True)

In [107]:
test_data.drop(columns={'Income_Verified','Gender','Home_Owner'},inplace=True)

In [108]:
train_data['Employment'] = train_data['Length_Employed'].str.replace('years',' ')
train_data['Employment'] = train_data['Employment'].str.replace('year',' ')
train_data['Employment'] = train_data['Employment'].str.replace('+',' ')
train_data['Employment'] = train_data['Employment'].str.replace('<',' ')

In [109]:
test_data['Employment'] = test_data['Length_Employed'].str.replace('years',' ')
test_data['Employment'] = test_data['Employment'].str.replace('year',' ')
test_data['Employment'] = test_data['Employment'].str.replace('+',' ')
test_data['Employment'] = test_data['Employment'].str.replace('<',' ')

In [110]:
X_train

,Loan_Amount_Requested,Annual_Income,Purpose_Of_Loan,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Female,Male,Mortgage,Own,Rent,Other,None,Verified_Income,Verified_Income_Source,Not_Verified,employment_bins
131196,0.217391,0.009472,2,0.274569,0.125,0.172222,0.223684,0.168831,0,1,1,0,0,0,0,0,0,1,1
9179,0.253623,0.005069,5,0.155789,0.250,0.172222,0.131579,0.071429,1,0,0,1,0,0,0,0,0,1,1
128579,0.124638,0.007871,2,0.539385,0.250,0.338889,0.197368,0.142857,0,1,0,0,1,0,0,0,0,1,3
87704,0.797101,0.029482,2,0.321830,0.000,0.172222,0.131579,0.116883,0,1,0,0,0,0,0,0,1,0,2
52653,0.014493,0.007871,9,0.341085,0.000,0.172222,0.092105,0.181818,0,1,1,0,0,0,0,0,0,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97639,0.130435,0.009205,4,0.689922,0.000,0.172222,0.263158,0.155844,0,1,1,0,0,0,0,0,0,1,2
95939,0.333333,0.011873,2,0.204801,0.000,0.172222,0.131579,0.071429,0,1,0,0,1,0,0,0,1,0,2
152315,0.134783,0.012807,1,0.328582,0.000,0.061111,0.092105,0.207792,1,0,0,0,0,0,0,0,0,1,3
117952,0.507246,0.006403,2,0.785446,0.000,0.172222,0.197368,0.136364,0,1,1,0,0,0,0,1,0,0,3


In [111]:
train_data['Employment'] = train_data['Employment'].astype(np.number)

In [112]:
test_data['Employment'] = test_data['Employment'].astype(np.number)

In [113]:
train_data['Loan_Amount_Requested'] = train_data['Loan_Amount_Requested'].str.replace(',','')

In [114]:
test_data['Loan_Amount_Requested'] = test_data['Loan_Amount_Requested'].str.replace(',','')

In [115]:
train_data['Loan_Amount_Requested'] = train_data['Loan_Amount_Requested'].astype(np.number)

In [116]:
test_data['Loan_Amount_Requested'] = test_data['Loan_Amount_Requested'].astype(np.number)

In [117]:
train_data['Employment'].fillna(train_data['Employment'].median(), inplace = True)

In [118]:
test_data['Employment'].fillna(test_data['Employment'].median(), inplace = True)

In [119]:
train_data['employment_bins'] = pd.cut(x=train_data['Employment'], bins=[0, 3, 7, 10],labels=[1,2,3])
train_data.drop(columns={'Employment','Length_Employed'},inplace=True)
train_data.rename(columns={'employment_bins':'Length_Employed'})
train_data.head()

,Loan_Amount_Requested,Annual_Income,Purpose_Of_Loan,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Interest_Rate,Female,Male,Mortgage,Own,Rent,Other,None,Verified_Income,Verified_Income_Source,Not_Verified,employment_bins
0,7000.0,68000.0,car,18.37,0,NaN,9,14,1,1,0,0,0,1,0,0,0,0,1,1
1,30000.0,NaN,debt_consolidation,14.93,0,17.0,12,24,3,1,0,1,0,0,0,0,1,0,0,2
2,24725.0,75566.4,debt_consolidation,15.88,0,NaN,12,16,3,0,1,1,0,0,0,0,0,1,0,2
3,16000.0,56160.0,debt_consolidation,14.34,3,NaN,16,22,3,0,1,0,0,0,0,0,0,1,0,1
4,17000.0,96000.0,debt_consolidation,22.17,1,NaN,19,30,1,1,0,0,1,0,0,0,0,1,0,3


In [120]:
train_data['Annual_Income'].fillna(train_data['Annual_Income'].median(), inplace = True)
train_data['Months_Since_Deliquency'].fillna(train_data['Months_Since_Deliquency'].median(), inplace = True)

In [121]:
lb_loan_purpose = LabelEncoder()
train_data['Purpose_Of_Loan'] = lb_loan_purpose.fit_transform(train_data['Purpose_Of_Loan'])

In [122]:
test_data['employment_bins'] = pd.cut(x=test_data['Employment'], bins=[0, 3, 7, 10],labels=[1,2,3])
test_data.drop(columns={'Employment','Length_Employed'},inplace=True)
test_data.rename(columns={'employment_bins':'Length_Employed'})
test_data.head()

,Loan_Amount_Requested,Annual_Income,Purpose_Of_Loan,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Female,Male,Mortgage,Own,Rent,Other,None,Verified_Income,Verified_Income_Source,Not_Verified,employment_bins
0,27500.0,129000.0,debt_consolidation,12.87,0,68.0,10,37,0,1,1,0,0,0,0,1,0,0,3
1,26000.0,110000.0,credit_card,11.37,0,NaN,6,23,0,1,0,0,0,0,0,0,0,1,3
2,6075.0,75000.0,debt_consolidation,6.83,2,NaN,5,20,0,1,0,0,1,0,0,1,0,0,1
3,12000.0,73000.0,debt_consolidation,7.76,0,NaN,6,8,0,1,1,0,0,0,0,0,1,0,3
4,35000.0,156000.0,debt_consolidation,9.62,0,26.0,9,21,0,1,1,0,0,0,0,0,0,1,1


In [123]:
X=train_data.drop(columns={'Interest_Rate'})
y=train_data['Interest_Rate']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [124]:
X_train.isnull().sum()

Loan_Amount_Requested      0
Annual_Income              0
Purpose_Of_Loan            0
Debt_To_Income             0
Inquiries_Last_6Mo         0
Months_Since_Deliquency    0
Number_Open_Accounts       0
Total_Accounts             0
Female                     0
Male                       0
Mortgage                   0
Own                        0
Rent                       0
Other                      0
None                       0
Verified_Income            0
Verified_Income_Source     0
Not_Verified               0
employment_bins            0
dtype: int64

In [125]:
# X_train['Annual_Income'].fillna(X_train['Annual_Income'].median(), inplace = True)
# X_test['Annual_Income'].fillna(X_test['Annual_Income'].median(), inplace = True)

In [126]:
test_data['Annual_Income'].fillna(test_data['Annual_Income'].median(), inplace = True)

In [33]:
# X_train['Months_Since_Deliquency'].fillna(X_train['Months_Since_Deliquency'].median(), inplace = True)
# X_test['Months_Since_Deliquency'].fillna(X_test['Months_Since_Deliquency'].median(), inplace = True)

In [127]:
test_data['Months_Since_Deliquency'].fillna(test_data['Months_Since_Deliquency'].median(), inplace = True)

In [34]:
#X_train.loc[pd.isna(X_train["employment_bins"]), :].index

In [128]:
X_test.isnull().sum()

Loan_Amount_Requested      0
Annual_Income              0
Purpose_Of_Loan            0
Debt_To_Income             0
Inquiries_Last_6Mo         0
Months_Since_Deliquency    0
Number_Open_Accounts       0
Total_Accounts             0
Female                     0
Male                       0
Mortgage                   0
Own                        0
Rent                       0
Other                      0
None                       0
Verified_Income            0
Verified_Income_Source     0
Not_Verified               0
employment_bins            0
dtype: int64

In [129]:
# lb_loan_purpose = LabelEncoder()
# X_train['Purpose_Of_Loan'] = lb_loan_purpose.fit_transform(X_train['Purpose_Of_Loan'])
# X_test['Purpose_Of_Loan'] = lb_loan_purpose.transform(X_test['Purpose_Of_Loan'])

In [130]:
lb1 = LabelEncoder()
test_data['Purpose_Of_Loan'] = lb1.fit_transform(test_data['Purpose_Of_Loan'])

In [131]:
X_train.head()

,Loan_Amount_Requested,Annual_Income,Purpose_Of_Loan,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Female,Male,Mortgage,Own,Rent,Other,None,Verified_Income,Verified_Income_Source,Not_Verified,employment_bins
131196,8000.0,75000.0,2,10.98,1,31.0,17,28,0,1,1,0,0,0,0,0,0,1,1
9179,9250.0,42000.0,5,6.23,2,31.0,10,13,1,0,0,1,0,0,0,0,0,1,1
128579,4800.0,63000.0,2,21.57,2,61.0,15,24,0,1,0,0,1,0,0,0,0,1,3
87704,28000.0,225000.0,2,12.87,0,31.0,10,20,0,1,0,0,0,0,0,0,1,0,2
52653,1000.0,63000.0,9,13.64,0,31.0,7,30,0,1,1,0,0,0,0,0,0,1,2


In [132]:
X_test.head()

,Loan_Amount_Requested,Annual_Income,Purpose_Of_Loan,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Female,Male,Mortgage,Own,Rent,Other,None,Verified_Income,Verified_Income_Source,Not_Verified,employment_bins
143221,10000.0,57600.0,2,18.46,1,31.0,14,21,1,0,1,0,0,0,0,0,0,1,3
40789,16000.0,63000.0,2,18.42,1,11.0,12,32,0,1,1,0,0,0,0,0,0,1,3
161602,14400.0,44000.0,2,32.43,0,72.0,7,30,0,1,0,1,0,0,0,1,0,0,2
22609,6500.0,60000.0,2,16.92,0,36.0,11,18,0,1,1,0,0,0,0,0,0,1,2
142297,13000.0,55000.0,2,17.22,1,31.0,9,29,0,1,0,0,1,0,0,0,1,0,3


In [133]:
train_data.select_dtypes(include=['number'])

,Loan_Amount_Requested,Annual_Income,Purpose_Of_Loan,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Interest_Rate,Female,Male,Mortgage,Own,Rent,Other,None,Verified_Income,Verified_Income_Source,Not_Verified
0,7000.0,68000.0,0,18.37,0,31.0,9,14,1,1,0,0,0,1,0,0,0,0,1
1,30000.0,63000.0,2,14.93,0,17.0,12,24,3,1,0,1,0,0,0,0,1,0,0
2,24725.0,75566.4,2,15.88,0,31.0,12,16,3,0,1,1,0,0,0,0,0,1,0
3,16000.0,56160.0,2,14.34,3,31.0,16,22,3,0,1,0,0,0,0,0,0,1,0
4,17000.0,96000.0,2,22.17,1,31.0,19,30,1,1,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164304,20725.0,63000.0,1,13.56,0,31.0,13,16,2,0,1,0,0,1,0,0,0,1,0
164305,6000.0,63000.0,11,9.12,0,36.0,13,30,2,0,1,0,1,0,0,0,0,1,0
164306,20250.0,63000.0,1,18.17,1,31.0,17,30,1,0,1,1,0,0,0,0,1,0,0
164307,10000.0,63000.0,6,18.75,1,48.0,16,62,3,1,0,0,0,1,0,0,0,0,1


In [134]:
num_columns = ['Loan_Amount_Requested','Annual_Income','Debt_To_Income','Inquiries_Last_6Mo','Months_Since_Deliquency','Number_Open_Accounts','Total_Accounts']
normalizer = MinMaxScaler()
#normalizer.fit(X_train[num_columns])
X_train[num_columns] = normalizer.fit_transform(X_train[num_columns])
X_test[num_columns] = normalizer.transform(X_test[num_columns])

print(X_train.head())
print(X_test.head())

        Loan_Amount_Requested  Annual_Income  Purpose_Of_Loan  Debt_To_Income  \
131196               0.217391       0.009472                2        0.274569   
9179                 0.253623       0.005069                5        0.155789   
128579               0.124638       0.007871                2        0.539385   
87704                0.797101       0.029482                2        0.321830   
52653                0.014493       0.007871                9        0.341085   

        Inquiries_Last_6Mo  Months_Since_Deliquency  Number_Open_Accounts  \
131196               0.125                 0.172222              0.223684   
9179                 0.250                 0.172222              0.131579   
128579               0.250                 0.338889              0.197368   
87704                0.000                 0.172222              0.131579   
52653                0.000                 0.172222              0.092105   

        Total_Accounts  Female  Male  Mortgage  Ow

In [135]:
normalizer1 = MinMaxScaler()
#normalizer.fit(X_train[num_columns])
test_data[num_columns] = normalizer.fit_transform(test_data[num_columns])

In [136]:
logreg = LogisticRegression(random_state=10) 

# fit on training data
logreg.fit(X_train , y_train) 
y_pred = logreg.predict(X_test)
# accuracy
f1_logreg = f1_score(y_pred, y_test, average='weighted') 
print("F1 score with Logistic Regression : ",f1_logreg)

F1 score with Logistic Regression :  0.5535908690023543


In [137]:
from sklearn.ensemble import RandomForestClassifier

# Code starts here

rf_clf_2 = RandomForestClassifier(n_estimators=100, min_samples_leaf=100 ,random_state=0)

rf_clf_2.fit(X_train, y_train)
y_pred = rf_clf_2.predict(X_test)
# accuracy with count vectorizer
f1_rf = f1_score(y_pred, y_test, average='weighted')

print("F1 Score with Random Forest: ", f1_rf)

F1 Score with Random Forest:  0.5545705200674984


In [138]:
dt = DecisionTreeClassifier(max_depth=1 , random_state=0)
dt.fit(X_train,y_train)
y_pred = dt.predict(X_test)
f1_dtclf = f1_score(y_pred, y_test, average='weighted')
print("F1 score with DT : ", f1_dtclf)



F1 score with DT :  0.5146212279175326


In [44]:
# svc_ova_cv = SVC(random_state=42, kernel='rbf', decision_function_shape='ova',C=10, gamma=0.1)
# svc_ova_cv.fit(X_train,y_train)
# y_pred = svc_ova_cv.predict(X_test)

# f1_svc = f1_score(y_pred, y_test, average='weighted')

# print("F1 score with SVM One Vs All : ", f1_svc)

In [139]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()


# fit on count vectorizer training data
nb.fit(X_train, y_train)

y_pred = nb.predict(X_test)
# accuracy with count vectorizer
f1_nb = f1_score(y_pred, y_test, average='weighted')

print("F1 Score with Naive Bayes using TFIDF: ", f1_nb)

F1 Score with Naive Bayes using TFIDF:  0.5358836372355018


In [140]:
gb = GradientBoostingClassifier(random_state =0)
gb.fit(X_train,y_train)
y_pred = gb.predict(X_test)

f1_gb = f1_score(y_pred, y_test, average='weighted')
print("F1 score with Gradient Boosting : ", f1_gb)

F1 score with Gradient Boosting :  0.5543892250184235


In [141]:
data_test=pd.read_csv('../123/test.csv')

In [142]:
data_test.head()

,Loan_ID,Loan_Amount_Requested,Length_Employed,Home_Owner,Annual_Income,Income_Verified,Purpose_Of_Loan,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Gender
0,10164310,"27,500",10+ years,Mortgage,129000.0,VERIFIED - income,debt_consolidation,12.87,0,68.0,10,37,Male
1,10164311,"26,000",10+ years,NaN,110000.0,not verified,credit_card,11.37,0,NaN,6,23,Male
2,10164312,"6,075",< 1 year,Rent,75000.0,VERIFIED - income,debt_consolidation,6.83,2,NaN,5,20,Male
3,10164313,"12,000",10+ years,Mortgage,73000.0,VERIFIED - income source,debt_consolidation,7.76,0,NaN,6,8,Male
4,10164314,"35,000",< 1 year,Mortgage,156000.0,not verified,debt_consolidation,9.62,0,26.0,9,21,Male


In [143]:
Id = data_test['Loan_ID']
prediction = rf_clf_2.predict(test_data)

prediction = pd.DataFrame(prediction,columns=['Interest_Rate'])
submission = pd.concat([Id,prediction['Interest_Rate']],1)
submission['Interest_Rate'].value_counts()
submission.to_csv('../123/submission.csv',index=False)

In [144]:
train_data['Interest_Rate'].value_counts()

2    70580
3    59923
1    33806
Name: Interest_Rate, dtype: int64